In [1]:
import requests
import re

def verify_pincode(address, pincode):
    # Make a request to the postalpincode.in API
    api_url = f"https://api.postalpincode.in/pincode/{pincode}"
    response = requests.get(api_url)
    
    if response.status_code == 200:
        data = response.json()
        
        if data and data[0]['Status'] == 'Success':
            
            text = data[0]['Message']
        
            # Useing regular expression to extract the number
            match = re.search(r'\d+', text)
            
            if match:
                number = int(match.group())
#                 print(number)
            else:
                print("No number found in the text.")
            
            for i in range(number):
                check_address = data[0]['PostOffice'][i]["Name"]
                if check_address in address:
                    return True
        return False

# Sample addresses and PIN codes for testing
correct_address_1 = "2nd Phase, 374/B, 80 Feet Rd, Mysore Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bengaluru, Karnataka 560050"
correct_address_2 = "374/B, 80 Feet Rd, State Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bangalore. 560050"
correct_address_3 = "2nd Phase, 374/B, 80 Feet Rd, Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bengaluru, Karnataka 560050"
incorrect_address_1 = "2nd Phase, 374/B, 80 Feet Rd, Mysore Bank Colony, Banashankari 3rd Stage, Srinivasa Nagar, Bengaluru, Karnataka 560095"
incorrect_address_2 = "Colony, Bengaluru, Karnataka 560050"

# Test cases
print(verify_pincode(correct_address_1, '560050'))  # Should return True
print(verify_pincode(correct_address_2, '560050'))  # Should return True
print(verify_pincode(correct_address_3, '560050'))  # Should return True
print(verify_pincode(incorrect_address_1, '560095'))  # Should return False
print(verify_pincode(incorrect_address_2, '560050'))  # Should return False


True
True
True
False
False


In [2]:
import mysql.connector

# Database connection parameters
config = {
    'user': 'rfamro',
    'host': 'mysql-rfam-public.ebi.ac.uk',
    'port': 4497,
    'database': 'Rfam',
    'raise_on_warnings': True,  # Optional: This setting will raise an exception for any warnings
}

# Connect to the database
try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    
    # Query 1: Number of tiger types
    cursor.execute("""
    SELECT COUNT(*) AS Number_of_Tiger_Types
    FROM taxonomy
    WHERE species LIKE '%Tiger%';
    """)

    # Fetch and print the result
    result = cursor.fetchone()
    print("Number of Tiger Types:", result[0])

    # Query 2: ncbi_id of the Sumatran Tiger
    cursor.execute("""
    SELECT ncbi_id
    FROM taxonomy
    WHERE species = 'Panthera tigris sumatrae';
    """)

    # Fetch and print the result
    result = cursor.fetchone()
    print("ncbi_id of Sumatran Tiger:", result)


    # Query 3
    cursor.execute("""
    SELECT t.species AS Rice_Species, MAX(rs.length) AS     
    Max_DNA_Sequence_Length
    FROM rfamseq AS rs
    JOIN taxonomy AS t ON rs.ncbi_id = t.ncbi_id
    WHERE t.species LIKE '%rice%'
    GROUP BY t.species
    ORDER BY Max_DNA_Sequence_Length DESC
    LIMIT 1;
    """)

    # Fetch and print the result
    result = cursor.fetchone()
    if result:
        rice_species = result[0]
        max_sequence_length = result[1]
        print(f"The rice species with the longest DNA sequence is '{rice_species}' with a length of {max_sequence_length}.")


    # Set the desired page and results per page
    page = 9
    results_per_page = 15
    # Calculate the offset for pagination
    offset = (page - 1) * results_per_page

    # Query 4. SQL query for pagination
    cursor.execute (f"""
    SELECT f.rfam_acc AS Family_Accession, f.description AS Family_Name,   
    MAX(rs.length) AS Max_DNA_Sequence_Length
    FROM family AS f
    JOIN full_region AS fr ON f.rfam_acc = fr.rfam_acc
    JOIN rfamseq AS rs ON fr.rfamseq_acc = rs.rfamseq_acc
    WHERE rs.length > 1000000
    GROUP BY f.rfam_acc, f.description
    ORDER BY Max_DNA_Sequence_Length DESC
    LIMIT {results_per_page} OFFSET {offset};
    """)

  # Fetch and print the result
    result = cursor.fetchall()

    if result:
            family_accession = result[0]
            family_name = result[1]
            max_sequence_length = result[2]
            print(f"Family Accession: {family_accession}, Family Name: {family_name}, Max DNA Sequence Length: {max_sequence_length}")


except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the cursor and database connection
    cursor.close()
    cnx.close()


Number of Tiger Types: 406
ncbi_id of Sumatran Tiger: None
The rice species with the longest DNA sequence is 'Cronobacter turicensis z3032' with a length of 4384463.
Family Accession: ('RF00278', 'Small nucleolar RNA SNORD50', 541556283), Family Name: ('RF00398', 'Small nucleolar RNA SNORA15', 541556283), Max DNA Sequence Length: ('RF00414', 'Small nucleolar RNA SNORA22', 541556283)


#### Q. Find all the columns that can be used to connect the tables in the given database.

Ans. To determine the columns that can be used to connect the tables in the given database, you need to identify common columns between the tables. Based on the provided table schema, here are the columns that can be used to connect the tables:

taxonomy and rfamseq can be connected using the ncbi_id column.

rfamseq and full_region can be connected using the rfamseq_acc column.

family and full_region can be connected using the rfam_acc column.

clan_membership and family can be connected using the rfam_acc column.

clan and clan_membership can be connected using the clan_acc column.

These columns serve as foreign keys to link the tables together based on their relationships in the schema. You can use these columns to perform joins between the tables to retrieve related information.